In [1]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from models.unet import unet
from tensorflow import keras
import tensorflow as tf
from utils.util import get_train_val_dataloader
from utils.callbacks import SelectCallbacks
from utils.loss import focal_loss
from utils.metrics import get_metrics

Segmentation Models: using `keras` framework.


In [35]:
!python examples/dubai.py

1044
1044


In [13]:
import json
import cv2
from examples.dubai import transform_data


# Note: this function must have the following structure
def read_img(directory, label=False, patch_idx=None):
    """
    Summary:
        read image with opencv and normalize the feature
    Arguments:
        directory (str): image path to read
        label (bool): the path is label path or not
        patch_idx (list): if want to extract patch from original image
    Return:
        numpy.array
    """


    if label:
        return cv2.cvtColor(cv2.imread(directory, 1), cv2.COLOR_BGR2RGB) # read and convert from BGR to RGB
    else:
        return cv2.imread(directory, 1)/255 # MinMaxScaler can be used for normalize


with open('config.json') as json_file:
    config = json.load(json_file)

config["transform_data"] = transform_data # this function will only apply on label
config["img_read_fn"] = read_img # this function execution will depend data whether its label or feature
train, val, config = get_train_val_dataloader(config)

metrics = list(get_metrics(config).values())

model = unet(config)
adam = keras.optimizers.Adam(learning_rate = config['learning_rate'])
loggers = SelectCallbacks(val, model, config)
model.compile(optimizer = adam, loss = focal_loss(), metrics = metrics)

history = model.fit(train,verbose = 1,epochs = 1,validation_data = val, shuffle = False, callbacks = loggers.get_callbacks())

Loading features and masks directories.....
train Example : 1044
valid Example : 130
348/348 [==============================] - 33s 71ms/step - loss: 0.1302 - my_mean_iou: 0.1056 - f1-score: 0.1944 - precision: 0.2784 - recall: 0.1757 - dice_coef_score: 0.3336 - val_loss: 0.1406 - val_my_mean_iou: 0.0852 - val_f1-score: 0.1583 - val_precision: 0.1908 - val_recall: 0.3708 - val_dice_coef_score: 0.3349
